In [1]:
import pandas as pd
import metadata_parser as mp
import requests
from bs4 import BeautifulSoup as bs
import re
import undetected_chromedriver as uc
import time

# Login Bibliopass

Bibliopass is a remote publisher library owned by the University of Turin. Each remote publisher library may have a different login interface/scheme so you need to modify this login code. If you want to use a Scopus subscription account, you also need to modify this login code. If you do not have access to log in to Scopus, you cannot use this code.

In [3]:
scopus_url = "https://www-scopus-com.bibliopass.unito.it/"

In [4]:
driver = uc.Chrome()

In [5]:
driver.get(scopus_url)
time.sleep(5)

In [ ]:
try:
    username = driver.find_element_by_id("username")
    password = driver.find_element_by_id("password")
    username.send_keys("your_username")
    password.send_keys("your_password")
    driver.find_element_by_name("_eventId_proceed").click()
except:
    pass

In [13]:
search_url = 'https://www-scopus-com.bibliopass.unito.it/results/results.uri?sort=plf-f&src=s&nlo=&nlr=&nls=&sid=45730a8677d84cbd18994d616df20e4e&sot=a&sdt=cl&cluster=scopubyr%2c%222022%22%2ct%2c%222021%22%2ct%2c%222020%22%2ct%2c%222019%22%2ct%2c%222018%22%2ct%2c%222017%22%2ct%2c%222016%22%2ct%2c%222015%22%2ct%2c%222014%22%2ct%2c%222013%22%2ct%2c%222012%22%2ct&sl=319&s=ABS%28%28%22Sentiment+Analysis%22%29+AND+%28%22Green%22+OR+%22Nature%22+OR+%22Environment%22+OR+%22Chemical%22+OR+%22Food%22+OR+%22Plant%22+OR+%22Organism%22+OR+%22Climate+Change%22+OR+%22Sustainability%22+OR+%22Sustainable%22+OR+%22Carbon%22+OR+%22Emission%22+OR+%22Waste%22+OR+%22Pollution%22+OR+%22Global+Warming%22%29+AND+%28%22Corpora%22+OR+%22Lexicon%22+OR+%22Model%22+OR+%22Algorithm%22+OR+%22Classifier%22%29%29&origin=resultslist&zone=leftSideBar&editSaveSearch=&txGid=fb75908e5bd39f76b5906f39158124fe'

In [14]:
driver.get(search_url)
time.sleep(5)

# Get Links

In [ ]:
# Get links from first pages
links = []
hrefs = driver.find_elements_by_xpath("//*[@class='searchArea']//*[@href]")
for a in hrefs:
    link = a.get_attribute('href')
    links.append(link)

# Get links from next pages
while True:
  try:
    driver.find_element_by_css_selector("[title='Next page'").click()
    time.sleep(5)
    hrefs = driver.find_elements_by_xpath("//*[@class='searchArea']//*[@href]")
    for a in hrefs:
        link = a.get_attribute('href')
        links.append(link)
  except:
    break

# Wrap Into Dataframe and Filtering Process

In [ ]:
# List filling
list_source = []
list_query = []
list_querylinks = []
[list_source.append("Scopus") for i in links]
[list_query.append("ALL") for i in links]
[list_querylinks.append(search_url) for i in links]

In [35]:
# Wrap into a dataframe
scopus = pd.DataFrame({'Source': list_source, 'Query': list_query, 'Query Links': list_querylinks, 'Paper Links': links})

In [37]:
# Drop links those are not a paper URL
scopus_links = scopus[scopus["Paper Links"].str.contains('record')==True]

In [39]:
# Save to Excel
scopus_links.to_excel("PaperLinks_Scopus.xlsx", index = False)

In [40]:
driver.close()